In [ ]:
!pip install sdv~=1.0.1

In [ ]:
from sdv.tabular import CTGAN
import pandas as pd
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from pickle import dump, load

In [ ]:
# https://www.kaggle.com/code/xiaxiaxu/predictmachinefailureinadvance/data
sensor = pd.read_csv("sensor.csv")

# only keeping cols w high var in pca analysis + machine status
data = sensor[
    ["timestamp", "sensor_25", "sensor_11", "sensor_36", "sensor_34", "machine_status"]
]

In [ ]:
# okay, what can our machine status be?

# data = data.convert_dtypes()
print(data.dtypes, "\n")
# i have no idea why i have to run this line twice for it to work.
data["timestamp"] = pd.to_datetime(data["timestamp"])

data.machine_status.unique()

In [ ]:
# okay, what can our machine status be?

# data = data.convert_dtypes()
print(data.dtypes, "\n")
# i have no idea why i have to run this line twice for it to work.
data["timestamp"] = pd.to_datetime(data["timestamp"])

data.machine_status.unique()

In [ ]:
data_around_failures = data.iloc[16000:26000]
len(data_around_failures)

In [ ]:
print(data_around_failures[data_around_failures.isna().any(axis=1)])

# we only have one row with NA, let's simply drop that.

data_around_failures.dropna(axis=0, inplace=True)

print(data_around_failures[data_around_failures.isna().any(axis=1)])

In [ ]:
scaler = MinMaxScaler((0, 1))
sensor_cols = ["sensor_25", "sensor_11", "sensor_36", "sensor_34"]

# scaling our data, then saving our scaler object for future use.
data_around_failures[sensor_cols] = scaler.fit_transform(
    data_around_failures[sensor_cols]
)

dump(scaler, open("ctgan_0_scaler.pkl", "wb"))

In [ ]:
data_around_failures.head()

In [ ]:
# finding number of times machine status is diff vars

n = data_around_failures.machine_status.value_counts()
print(n)

In [ ]:
# going to try to keep every 2 minutes rather than every minute

thin_data = data_around_failures[data_around_failures.index % 2 == 0]
thin_data

In [ ]:
# thin_data = thin_data.convert_dtypes()

print(thin_data.dtypes, "\n")

In [ ]:
model = CTGAN(
    verbose=True,
    epochs=500,
    batch_size=300,
    log_frequency=False,
    embedding_dim=256,
    discriminator_steps=3,
)
import time

In [ ]:
tic = time.perf_counter()
model.fit(thin_data)
toc = time.perf_counter()

In [ ]:
print(f"Trained in {toc - tic:0.4f} seconds")
model.save("ctgan_model_new_params.pkl")

In [ ]:
new_data = model.sample(num_rows=3000)

In [ ]:
new_data = new_data.sort_values(by=["timestamp"])
new_data

In [ ]:
import matplotlib.pyplot as plt

# y='sensor_11'
# thin_data.plot(y='sensor_11')
thin_data.set_index("timestamp").plot(y="sensor_34")
thin_data.set_index("timestamp").plot()

In [ ]:
new_data.set_index("timestamp").plot(y="sensor_34")
new_data.set_index("timestamp").plot()
# new_data.plot(y='sensor_11')

In [ ]:
dups_time = new_data.pivot_table(columns=["timestamp"], aggfunc="size")
print(dups_time.sort_values())

In [ ]:
no_dups = new_data.groupby("timestamp").mean().reset_index()
no_dups

In [ ]:
new_data.set_index("timestamp").plot(y="sensor_34")
new_data.set_index("timestamp").plot()

I tested this model with a wide variety of hyperparameters on Monday 7/11, with no luck. I think that it's safe to give up on SDV for our purposes. 